First attempt at web scraping following: https://towardsdatascience.com/data-science-skills-web-scraping-using-python-d1a85ef607ed

In [44]:
from bs4 import BeautifulSoup
import urllib.request
import csv

In [46]:
urlpage='https://www.fasttrack.co.uk/league-tables/tech-track-100/league-table/'
page = urllib.request.urlopen(urlpage)
soup = BeautifulSoup(page, 'html.parser')

In [45]:
print(soup)

<!DOCTYPE html>

<!--[if lt IE 7 ]> <html class="ie ie6 no-js" lang="en-GB"> <![endif]-->
<!--[if IE 7 ]>    <html class="ie ie7 no-js" lang="en-GB"> <![endif]-->
<!--[if IE 8 ]>    <html class="ie ie8 no-js" lang="en-GB"> <![endif]-->
<!--[if IE 9 ]>    <html class="ie ie9 no-js" lang="en-GB"> <![endif]-->
<!--[if gt IE 9]><!-->
<html class="no-js" lang="en-GB">
<!--<![endif]-->
<!-- the "no-js" class is for Modernizr. -->
<head id="live2-fasttrack-com"><link data-minify="1" href="https://www.fasttrack.co.uk/wp-content/cache/min/1/a92c3e8a69c819d3455d1b6e3cd87bb4.css" rel="stylesheet"/>
<meta charset="utf-8"/>
<!-- Always force latest IE rendering engine (even in intranet) & Chrome Frame -->
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>
        Plan.com - Fast Track    </title>
<meta content="Plan.com - Fast Track" name="title"/>
<meta content="" name="description"/>
<meta content="" name="keyword"/>
<meta content="ZYBMlezF37FAeJwYQIBdbJtogbNm9zRcb4fCelYavj8"

In [48]:
table = soup.find('table',attrs={'class':'tableSorter'})
results = table.find_all('tr')
print('Number of results:',len(results))

Number of results: 101


In [15]:
results[:2]

[<tr>
 <th>Rank</th>
 <th>Company</th>
 <th class="">Location</th>
 <th class="no-word-wrap">Year end</th>
 <th class="" style="text-align:right;">Annual sales rise over 3 years</th>
 <th class="" style="text-align:right;">Latest sales £000s</th>
 <th class="" style="text-align:right;">Staff</th>
 <th class="">Comment</th>
 <!--				<th>FYE</th>-->
 </tr>, <tr>
 <td>1</td>
 <td><a href="https://www.fasttrack.co.uk/company_profile/plan-com/"><span class="company-name">Plan.com</span></a>Communications provider</td>
 <td>Isle of Man</td>
 <td>Sep-17</td>
 <td style="text-align:right;">364.38%</td>
 <td style="text-align:right;">*35,418</td>
 <td style="text-align:right;">90</td>
 <td>About 650 partners use its telecoms platform to support more than 100,000 UK business customers</td>
 <!--						<td>Sep-17</td>-->
 </tr>]

In [54]:
rows = []
rows.append(['Rank','Company Name','Webpage','Description','Location', 'Year End','3 Year Sales Growth','Sales £000s','Staff','Comments'])
print(rows)

for result in results:
    data = result.find_all('td')

    if len(data) == 0:

        continue
    rank = data[0].getText()
    company = data[1].getText()
    location = data[2].getText()
    yearend = data[3].getText()
    salesgrowth = data[4].getText()
    sales = data[5].getText()
    staff = data[6].getText()
    comments = data[7].getText()

    name = data[1].find('span',attrs={'class':'company-name'}).getText()
    description = company.replace(name,'')
    sales = sales.strip('*').strip('†').replace(',','')

    url = data[1].find('a').get('href')
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page,'html.parser')

    try:
        tableRow=soup.find('table').find_all('tr')[-1] #last entry?
        webpage = tableRow.find('a').get('href')
    except:
        webpage = None

    rows.append([rank,name,webpage,description,location,yearend,salesgrowth,sales,staff,comments])
        
print(rows)
        

[['Rank', 'Company Name', 'Webpage', 'Description', 'Location', 'Year End', '3 Year Sales Growth', 'Sales £000s', 'Staff', 'Comments']]
[['Rank', 'Company Name', 'Webpage', 'Description', 'Location', 'Year End', '3 Year Sales Growth', 'Sales £000s', 'Staff', 'Comments'], ['1', 'Plan.com', 'http://www.plan.com', 'Communications provider', 'Isle of Man', 'Sep-17', '364.38%', '35418', '90', 'About 650 partners use its telecoms platform to support more than 100,000 UK business customers'], ['2', 'PsiOxus', 'http://www.psioxus.com', 'Biotechnology developer', 'Oxfordshire', 'Dec-17', '311.67%', '53136', '54', 'Received a $15m milestone payment from its development partner Bristol-Myers Squibb in December'], ['3', 'CensorNet', 'http://www.censornet.com', 'Cloud security software developer', 'Basingstoke', 'Dec-17', '210.17%', '7535', '77', 'Has more than 4,000 customers, including McDonald’s in France and the European Central Bank'], ['4', 'thoughtonomy', 'http://www.thoughtonomy.com', 'Auto

In [55]:
with open('techtrack100.csv','w',newline='') as f_output:
    csv_output = csv.writer(f_output)
    csv_output.writerows(rows)